In [31]:
# !pip install openai
# !pip install langchain
# !pip install pyspark
!pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26800 sha256=9a0e2eaf80b4ff010847e6f3434b2f64bac33e669f00cbcb47edaf204542bd01
  Stored in directory: c:\users\manas\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [32]:
import openai
import langchain
import os
from langchain.llms import OpenAI
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pandasql import sqldf

In [21]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234' -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"

In [22]:
s1 = SparkSession.builder.config("k1", "v1").getOrCreate()
s1.conf.get("k1") == "v1"

RuntimeError: Java gateway process exited before sending its port number

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

C:\Users\Manas\anaconda3\Lib\site-packages\langchain\__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
C:\Users\Manas\anaconda3\Lib\site-packages\langchain\__init__.py:24: UserWarning: Importing LLMChain from langchain root module is no longer supported.
  warnings.warn(


In [7]:
termination_data=pd.read_csv('MFG10YearTerminationData.csv')
termination_data

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49648,8258,12/1/2015 0:00,5/28/1994,8/19/2013,12/30/2015,21,2,Valemount,Dairy,Dairy Person,34,M,Male,Layoff,Involuntary,2015,TERMINATED,STORES
49649,8264,8/1/2013 0:00,6/13/1994,8/27/2013,8/30/2013,19,0,Vancouver,Customer Service,Cashier,44,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES
49650,8279,12/1/2015 0:00,7/18/1994,9/15/2013,12/30/2015,21,2,White Rock,Customer Service,Cashier,39,F,Female,Layoff,Involuntary,2015,TERMINATED,STORES
49651,8296,12/1/2013 0:00,9/2/1994,10/9/2013,12/31/2013,19,0,Kelowna,Customer Service,Cashier,16,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES


In [8]:
termination_data.columns

Index(['EmployeeID', 'recorddate_key', 'birthdate_key', 'orighiredate_key',
       'terminationdate_key', 'age', 'length_of_service', 'city_name',
       'department_name', 'job_title', 'store_name', 'gender_short',
       'gender_full', 'termreason_desc', 'termtype_desc', 'STATUS_YEAR',
       'STATUS', 'BUSINESS_UNIT'],
      dtype='object')

In [39]:
meta_data_json={
    'table_name': 'termination_data',
    'column_names':{
        'EmployeeID':'The employee id', 
        'recorddate_key': 'The record date of the employee',
        'birthdate_key': 'The birth date of the employee', 
        'orighiredate_key': 'The date the employee started working',
        'terminationdate_key': 'The date the employee was terminated from the job', 
        'age': 'The age of the employee', 
        'length_of_service': 'The duration the employee served in the organization', 
        'city_name': 'The city of the employee',
        'department_name': 'The department the employee was working in', 
        'job_title': 'The job title of the employee', 
        'store_name': 'The store that the employee worked in', 
        'gender_short': 'The gender of the employee. It is stored in short form',
        'gender_full': 'The gender of the employee', 
        'termreason_desc': 'The reason the employee was terminated', 
        'termtype_desc': 'the description of the term, the values are: "Not Applicable", "Voluntary"', 
        'STATUS_YEAR': 'The year',
        'STATUS': 'The current status of the employee', 
        'BUSINESS_UNIT': 'The business unit of the employee'
    }
}

In [ ]:
Chat GPT Response

To retrieve the total number of employees who left voluntarily in the year 2006 from the "termination_data" table, you can use the following SQL script:

```sql
SELECT COUNT(*) AS TotalVoluntaryDepartures
FROM termination_data
WHERE STATUS_YEAR = 2006
  AND termreason_desc = 'Voluntary'
```

In this SQL script:

- We use the `SELECT` statement to retrieve data from the table.
- We use the `COUNT(*)` function to count the number of rows that match the specified conditions.
- We specify the table name as "termination_data" from which we want to retrieve data.
- We use the `WHERE` clause to filter the rows where `STATUS_YEAR` is equal to 2006 (i.e., employees in the year 2006) and `termreason_desc` is 'Voluntary' (i.e., employees who left voluntarily).
- We alias the count result as "TotalVoluntaryDepartures" to make the output more readable.

This script will return the total number of employees who left voluntarily in the year 2006 from the "termination_data" table.

In [42]:
sqldf("""

SELECT COUNT(*) AS TotalVoluntaryDepartures
FROM termination_data
WHERE STATUS_YEAR = '2006'
  AND termtype_desc = 'Voluntary'


""")

,TotalVoluntaryDepartures
0,134
